In [1]:
import pandas as pd
import scipy.io
import re
import sys
import warnings
import pickle

In [2]:
from mvpa2.datasets.mri import fmri_dataset

Failed to import duecredit due to No module named 'duecredit'
/home/bsmith16/.conda/envs/py3_mvpa/lib/python3.8/site-packages/mvpa2/datasets/base.py:465: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def __init__(self, shape=None, sid=None, fid=None, dtype=np.float):


In [3]:
class BehavioralDataNotFoundForBrainDataException(Exception):
    """Behavioral data could not be matched to a subject."""
    pass

Replicating earlier work on mvpa. Try not to overly complicate it--the main point is just to verify we get similar results on a different package to validate prior work. ANd we are primarily interested in validating the very high cross-validation results I got with nltools. Should aim for readable code.

In [4]:
from mvpa2 import *

In [5]:


nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

In [6]:
brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_conditions_84subs_correct_cond.pkl'
results_filepath=ml_data_folderpath + "/SST/mvpa_tt_res_v3_conditions_84subs_twoclasses_pfcmask.pkl"

In [7]:
include_exclude_list = pd.read_csv("../nsc_subject_exclusions.csv")

In [8]:
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20211027T173724.csv")
test_train_df_raw = test_train_df_raw.merge(include_exclude_list[include_exclude_list.Task=='SST'],left_on='sub_label',right_on='SubjectId',how='left')
test_train_df_raw.loc[test_train_df_raw.Include.isna(),'Include'] = True
test_train_df = test_train_df_raw[test_train_df_raw.Include==True]
#exclude_subjects = ['DEV061','DEV185','DEV187','DEV189','DEV190','DEV192','DEV198','DEV203','DEV220','DEV221']
exclude_subjects = ['DEV098', 'DEV153', 'DEV154', 'DEV163', 'DEV170', 'DEV174', 'DEV181', 'DEV188', 'DEV200', 'DEV207', 'DEV208']
validation_exclude_subject = ['DEV098']
valiation_subjs = test_train_df.loc[test_train_df.SplitGroup=='Holdout','sub_label'].tolist()#only get the train subjects; ignore those previously marked hold-out

In [9]:
valiation_subjs_selected = [ts for ts in valiation_subjs if (ts not in exclude_subjects)]

In [10]:
", ".join(valiation_subjs_selected)

'DEV004, DEV008, DEV031, DEV033, DEV037, DEV038, DEV044, DEV045, DEV054, DEV065, DEV066, DEV070, DEV080, DEV081, DEV091, DEV007, DEV063, DEV072, DEV075, DEV078, DEV082, DEV088, DEV090, DEV093, DEV094, DEV095, DEV096, DEV099, DEV100, DEV102, DEV103, DEV106, DEV108, DEV110, DEV111, DEV112, DEV113, DEV114, DEV116, DEV117, DEV118, DEV119, DEV120, DEV121, DEV122, DEV123, DEV127, DEV131, DEV133, DEV137, DEV138, DEV144, DEV147, DEV150, DEV155, DEV156, DEV161, DEV166, DEV167, DEV168, DEV172, DEV176, DEV178, DEV179, DEV180, DEV182, DEV183, DEV191, DEV193, DEV194, DEV195, DEV196, DEV199, DEV201, DEV202, DEV204, DEV205, DEV209, DEV211, DEV219, DEV222, DEV223, DEV224, DEV225'

In [11]:
individual_differences = pd.read_csv(ml_data_folderpath + "/data_by_ppt.csv")
individual_differences = individual_differences.rename(columns={'SID':'subject'})
individual_differences['wave']=1

In [12]:
import sys

sys.path.append(os.path.abspath("../../ml/"))

We probably actually want to start the pipeline from the betas rather than loading from pickle. to be continued...

In [13]:
from mvpa_pipeline_utils import get_Brain_Data_betas_as_mvpa_for_sub, import_beta_series_pymvpa2, sa_to_df

In [14]:
import mvpa2.datasets 

In [15]:
my_test_data =import_beta_series_pymvpa2(
    valiation_subjs_selected[0:2],'conditions',
    out_folder = ml_data_folderpath + "/SST/val_set_incomplete_",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond_',
    beta_processing_args = {
        'contrast_condition_1':'CorrectStop',
        'contrast_condition_2':'CorrectGo',
        'spatially_concatenate':False
    }
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV004
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV004/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV004/beta_0001.nii
 * Please note: warnings are printed only once, but underlying problem might occur many times *
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV004     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV004     1  
DEV008
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV008/SPM.mat
.
.
...verified th

In [16]:
sa_to_df(my_test_data.sa)

,time_indices,time_coords,condition_index,condition_label,raw_beta_description,beta,subject,wave,cancer_promoting_minus_preventing_FCI,cancer_promoting_minus_preventing_FFQ,...,WTP_unhealthy,WTP_unhealthy_minus_healthy,SRHI_healthy_minus_unhealthy,RTFS_f1_minus_f2,IPAQ_walkingMETminutes,IPAQ_moderateMETminutes,IPAQ_vigorousMETminutes,IPAQ_total_METminutes,IPAQ_MET_kCal,birthsex_factor
0,0,0.0,"0, 1","CorrectGo, CorrectStop","spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1), s...","beta_0001.nii, beta_0002.nii",DEV004,1,0.113978,-0.698077,...,2.000000,-0.190476,-11.0,NaN,0.0,1440.0,240.0,1680.0,NaN,Female
1,0,0.0,"0, 1","CorrectGo, CorrectStop","spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1), s...","beta_0001.nii, beta_0002.nii",DEV008,1,-0.170430,-0.932692,...,1.806452,-1.256048,32.0,NaN,990.0,360.0,320.0,1670.0,NaN,Female


In [17]:
my_test_data =import_beta_series_pymvpa2(
    valiation_subjs_selected[0:10],'conditions',
    out_folder = ml_data_folderpath + "/SST/val_set_incomplete_",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond',
    beta_processing_args = {
        'spatially_concatenate':False
    }
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV004
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV004/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV004/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV004     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV004     1  
DEV008
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV008/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series bet

In [18]:
my_test_data =import_beta_series_pymvpa2(
    valiation_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/val_set_incomplete_",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond',
    beta_processing_args = {
        'spatially_concatenate':False
    }
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV004
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV004/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV004/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV004     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV004     1  
DEV007
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV007/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series bet

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV066     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV066     1  
DEV070
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV070/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV070/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV070     1  
1  spm_spm:beta (0002) - Sn(1)

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV095     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV095     1  
DEV096
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV096/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV096/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV096     1  
1  spm_spm:beta (0002) - Sn(1)

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV114     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV114     1  
DEV116
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV116/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV116/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV116     1  
1  spm_spm:beta (0002) - Sn(1)

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV137     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV137     1  
DEV138
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV138/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV138/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV138     1  
1  spm_spm:beta (0002) - Sn(1)

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV176     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV176     1  
DEV178
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV178/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV178/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV178     1  
1  spm_spm:beta (0002) - Sn(1)

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV201     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV201     1  
DEV202
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV202/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV202/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV202     1  
1  spm_spm:beta (0002) - Sn(1)

In [19]:
my_test_data =import_beta_series_pymvpa2(
    valiation_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/val_set_incomplete_",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_cor_stop_minus_go_contrast',
    beta_processing_args = {
        'contrast_condition_1':'CorrectStop',
        'contrast_condition_2':'CorrectGo',
        'spatially_concatenate':False
    }
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV004
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV004/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV004/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV004     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV004     1  
DEV007
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV007/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series bet

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV066     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV066     1  
DEV070
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV070/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV070/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV070     1  
1  spm_spm:beta (0002) - Sn(1)

.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV095/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV095     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV095     1  
DEV096
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV096/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV096/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo 

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV114     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV114     1  
DEV116
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV116/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV116/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV116     1  
1  spm_spm:beta (0002) - Sn(1)

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV137     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV137     1  
DEV138
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV138/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV138/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV138     1  
1  spm_spm:beta (0002) - Sn(1)

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV176     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV176     1  
DEV178
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV178/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV178/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV178     1  
1  spm_spm:beta (0002) - Sn(1)

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV201     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV201     1  
DEV202
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV202/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV202/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV202     1  
1  spm_spm:beta (0002) - Sn(1)

In [20]:
my_test_data.a['get_sa_as_df'] = lambda: sa_to_df(my_test_data.sa)